In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [4]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# For a given dataset (as pandas dataframe) and a given model year, finds matching model data (depth-matched)

In [5]:
#imports
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seawater
import netCDF4 as nc
from scipy import stats

from numpy import *
from scipy import *

import netCDF4 as nc
import numpy as np
import scipy as sp
import cmocean
import glob
import seawater
import arrow
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv('./Datasets/combined_fromdf.csv')
dsetname = 'COMB'

dic = df['dic'].to_numpy()
alk = df['alk'].to_numpy()
dic_qf = df['dic_qf'].to_numpy()
alk_qf = df['alk_qf'].to_numpy()
lat = df['lat'].to_numpy()
lon = df['lon'].to_numpy()
S = df['S'].to_numpy()
T = df['T'].to_numpy()
P = df['P'].to_numpy()
mon = df['mon'].to_numpy()
day = df['day'].to_numpy()

# infil = np.loadtxt('./Datasets/grl2016_nu.txt')

# crid= infil[:,0]
# ln = infil[:,2]
# stn = infil[:,3]
# mon = infil[:,4]
# day = infil[:,5]
# lat_or = infil[:,6]
# lon_or = infil[:,7]
# lat = infil[:,6]
# lon = infil[:,7]
# P = infil[:,8]
# T = infil[:,9]
# S = infil[:,10]
# ox = infil[:,11]
# ox_qf = infil[:,12]
# dic = infil[:,13]
# alk = infil[:,15]
# dic_qf = infil[:,14]
# alk_qf = infil[:,16]
# no3 = infil[:,17]
# no3_qf = infil[:,18]
# si = infil[:,19]
# si_qf = infil[:, 20]
# po4 = infil[:,21]
# po4_qf = infil[:, 22]






In [8]:
##add model coords
obs_i = np.zeros_like(alk)
obs_j = np.zeros_like(alk)
moddepth = np.zeros_like(alk)
depthdist_modobs = np.zeros_like(alk)

mesh = nc.Dataset('/data/tjarniko/MEOPAR/grid/bathymetry_201702.nc')
grid = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')
nav_lon = mesh.variables['nav_lon'][:]
nav_lat = mesh.variables['nav_lat'][:]
bathy = mesh.variables['Bathymetry'][:]
depth = grid.variables['gdept_1d'][:]

dist = []
for w in range(0,len(lat)):
    j, i = geo_tools.find_closest_model_point(lon[w],50.116,nav_lon,nav_lat)
    obs_j[w] = j
    obs_i[w] = i
    close = np.abs(depth-P[w])
    # smallest
    depthdist_modobs[w] = (np.min(close))
    moddepth[w] = (np.argmin(close))
    

In [9]:
mask = (((dic_qf==2) | (dic_qf==6)) & \
        ((alk_qf==2) | (alk_qf==6)) & \
        (dic > 0) & (alk >0))

totobs = (np.size(dic[mask]))

### get accessable subset


dens = seawater.dens(S,T,P)
sal_conv = S * 35.16504/35
# CONVERT FROM UMOL TO MMOL/M3
ta_conv=alk*dens/1000
dic_conv=dic*dens/1000
sal_conv = S * 35.16504/35

### subset, converted to model units
dic_SSconv = dic_conv[mask]
ta_SSconv = ta_conv[mask]
sal_SSconv = sal_conv[mask]
depth_SSconv = P[mask]; lat_SSconv = lat[mask]; lon_SSconv = lon[mask]
mon_SSconv = mon[mask]; day_SSconv = day[mask]; i_SSconv = obs_i[mask].astype(int);
j_SSconv = (obs_j[mask].astype(int))
moddepth_SSconv = moddepth[mask]; depthdist_modobs_SSconv = depthdist_modobs[mask]


In [21]:
np.min(moddepth)

0.0

In [10]:
def find_model_dat_bydepth(w, yr):
    
    sal_marker = 1
    lb_loc = np.nan; ub_loc = np.nan
    TA_int = np.nan; DIC_int = np.nan; temp_int = np.nan
    
    #find data file 
    mon = mon_SSconv[w]; day = day_SSconv[w]
    tdate = f'{yr}-{int(mon)}-{int(day)}'
    td = arrow.get(tdate)
    ymd = td.format('YYYYMMDD')
    #print(ymd)
    
    #open the relevant carp and grid files
    ptd = f'/data/tjarniko/results/hindcast.201905_dayavg_DIC-TA-T-S/carp_1d_{ymd}.nc'
    ptd2 = glob.glob(ptd)
    carpnc = nc.Dataset(ptd2[0])
    ptd = f'/data/tjarniko/results/hindcast.201905_dayavg_DIC-TA-T-S/grid_1d_{ymd}.nc'
    ptd2 = glob.glob(ptd)
    gridnc = nc.Dataset(ptd2[0])
    
    # extract salinity, temperature, etc
    salvec = gridnc['model_output']['SAL'][:,j_SSconv[w],i_SSconv[w]]
    salvec[salvec == 0] = np.nan
    
    tempvec = gridnc['model_output']['TEMP'][:,j_SSconv[w],i_SSconv[w]]
    tempvec[tempvec == 0] = np.nan

    TAvec = carpnc['model_output']['TA'][:,j_SSconv[w],i_SSconv[w]]
    TAvec[TAvec == 0] = np.nan

    DICvec = carpnc['model_output']['DIC'][:,j_SSconv[w],i_SSconv[w]]
    DICvec[DICvec == 0] = np.nan
    
#     print(moddepth_SSconv[w])
    depthind = int(moddepth_SSconv[w])
    
    DIC_int = DICvec[depthind]
    TA_int = TAvec[depthind]
    temp_int = tempvec[depthind]
    sal_int = salvec[depthind]
        
    gridnc.close(); carpnc.close()
    
    return TA_int, DIC_int, temp_int, sal_int
    



In [26]:
def find_model_dat_bydepth_BR3(w, yr):
    
    sal_marker = 1
    lb_loc = np.nan; ub_loc = np.nan
    TA_int = np.nan; DIC_int = np.nan; temp_int = np.nan
    
    #find data file 
    mon = mon_SSconv[w]; day = day_SSconv[w]
    tdate = f'{yr}-{int(mon)}-{int(day)}'
    td = arrow.get(tdate)
    ymd = td.format('YYYYMMDD')
    #print(ymd)
    
    #open the relevant carp and grid files
    ptd = f'/data/tjarniko/results/BASERUN_EXP/MAIN/BR_3rd_2015/ncs/SKOG_1d_*carp*{ymd}.nc'
    ptd2 = glob.glob(ptd)
    carpnc = nc.Dataset(ptd2[0])
    #print(carpnc)
    ptd = f'/data/tjarniko/results/BASERUN_EXP/MAIN/BR_3rd_2015/ncs/SKOG_1d_*grid_T*{ymd}.nc'
    ptd2 = glob.glob(ptd)
    gridnc = nc.Dataset(ptd2[0])
    
    # extract salinity, temperature, etc
    salvec = gridnc['vosaline'][0,:,j_SSconv[w],i_SSconv[w]]
    salvec[salvec == 0] = np.nan
    
    tempvec = gridnc['votemper'][0,:,j_SSconv[w],i_SSconv[w]]
    tempvec[tempvec == 0] = np.nan

    TAvec = carpnc['total_alkalinity'][0,:,j_SSconv[w],i_SSconv[w]]
    TAvec[TAvec == 0] = np.nan

    DICvec = carpnc['dissolved_inorganic_carbon'][0,:,j_SSconv[w],i_SSconv[w]]
    DICvec[DICvec == 0] = np.nan
    
#     print(moddepth_SSconv[w])
    depthind = int(moddepth_SSconv[w])
    
    DIC_int = DICvec[depthind]
    TA_int = TAvec[depthind]
    temp_int = tempvec[depthind]
    sal_int = salvec[depthind]
        
    gridnc.close(); carpnc.close()
    
    return TA_int, DIC_int, temp_int, sal_int
    



In [27]:

for yr in range(2015,2016):
    print(yr)
    ncname = f'./ncs/YR{yr}BR3_{dsetname}_depthspace_modeval.nc'
    print(totobs)
    lb_loc_ar = np.zeros(totobs)
    ub_loc_ar = np.zeros(totobs)
    sal_marker_ar = np.zeros(totobs)
    TA_int_ar = np.zeros(totobs)
    DIC_int_ar = np.zeros(totobs)
    temp_int_ar = np.zeros(totobs)
    sal_int_ar = np.zeros(totobs)

    for i in range(0,totobs):
        if i%10 == 0: 
            print(i)
        TA_int_ar[i], DIC_int_ar[i], temp_int_ar[i], sal_int_ar[i] = find_model_dat_bydepth_BR3(i,yr)

    # print(TA_int_ar)
    f = nc.Dataset(ncname,'w', format='NETCDF4') #'w' stands for write
    g = f.createGroup('model_output')
    g.createDimension('ref', totobs)
    ts = g.createVariable('model_j','f4',('ref'))
    ts[:] = j_SSconv[:]
    ts = g.createVariable('model_i','f4',('ref'))
    ts[:] = i_SSconv[:]
    ts = g.createVariable('model_d','f4',('ref'))
    ts[:] = moddepth_SSconv[:]
    ts3 = g.createVariable('TA_int','f4',('ref'))
    ts3[:] = TA_int_ar[:]
    ts3 = g.createVariable('DIC_int','f4',('ref'))
    ts3[:] = DIC_int_ar[:]
    ts3 = g.createVariable('temp_int','f4',('ref'))
    ts3[:] = temp_int_ar[:]
    ts3 = g.createVariable('sal_int','f4',('ref'))
    ts3[:] = sal_int_ar[:]
    f.close()

2015
572
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570


In [33]:


for yr in range(2013,2018):
    print(yr)
    ncname = f'./ncs/YR{yr}_{dsetname}_depthspace_modeval.nc'

    lb_loc_ar = np.zeros(totobs)
    ub_loc_ar = np.zeros(totobs)
    sal_marker_ar = np.zeros(totobs)
    TA_int_ar = np.zeros(totobs)
    DIC_int_ar = np.zeros(totobs)
    temp_int_ar = np.zeros(totobs)
    sal_int_ar = np.zeros(totobs)

    for i in range(0,totobs):
        if i%10 == 0: 
            print(i)
        TA_int_ar[i], DIC_int_ar[i], temp_int_ar[i], sal_int_ar[i] = find_model_dat_bydepth(i,yr)

    # print(TA_int_ar)
    f = nc.Dataset(ncname,'w', format='NETCDF4') #'w' stands for write
    g = f.createGroup('model_output')
    g.createDimension('ref', totobs)
    ts = g.createVariable('model_j','f4',('ref'))
    ts[:] = j_SSconv[:]
    ts = g.createVariable('model_i','f4',('ref'))
    ts[:] = i_SSconv[:]
    ts = g.createVariable('model_d','f4',('ref'))
    ts[:] = moddepth_SSconv[:]
    ts3 = g.createVariable('TA_int','f4',('ref'))
    ts3[:] = TA_int_ar[:]
    ts3 = g.createVariable('DIC_int','f4',('ref'))
    ts3[:] = DIC_int_ar[:]
    ts3 = g.createVariable('temp_int','f4',('ref'))
    ts3[:] = temp_int_ar[:]
    ts3 = g.createVariable('sal_int','f4',('ref'))
    ts3[:] = sal_int_ar[:]
    f.close()

2013
0
10
20
30
40
50
60
70
2014
0
10
20
30
40
50
60
70
2015
0
10
20
30
40
50
60
70
2016
0
10
20
30
40
50
60
70
2017
0
10
20
30
40
50
60
70


In [34]:
# lb_loc_ar = np.zeros(425)
# ub_loc_ar = np.zeros(425)
# sal_marker_ar = np.zeros(425)
# TA_int_ar = np.zeros(425)
# DIC_int_ar = np.zeros(425)
# temp_int_ar = np.zeros(425)

In [35]:
# ncname = 'YR2015_salspace_modeval.nc'
# f = nc.Dataset(ncname,'w', format='NETCDF4') #'w' stands for write
# g = f.createGroup('model_output')
# g.createDimension('ref', 425)
# ts = g.createVariable('model_j','f4',('ref'))
# ts[:] = j_SSconv[:]
# ts = g.createVariable('model_i','f4',('ref'))
# ts[:] = i_SSconv[:]
# ts = g.createVariable('lb_loc','f4',('ref'))
# ts[:] = lb_loc_ar[:]
# ts2 = g.createVariable('ub_loc','f4',('ref'))
# ts2[:] = ub_loc_ar[:]
# ts3 = g.createVariable('sal_marker','f4',('ref'))
# ts3[:] = sal_marker_ar[:]
# ts3 = g.createVariable('TA_int','f4',('ref'))
# ts3[:] = TA_int_ar[:]
# ts3 = g.createVariable('DIC_int','f4',('ref'))
# ts3[:] = DIC_int_ar[:]
# ts3 = g.createVariable('temp_int','f4',('ref'))
# ts3[:] = temp_int_ar[:]
# f.close()

In [36]:
# ### subset, converted to model units
# dic_SSconv = dic_conv[mask]
# ta_SSconv = ta_conv[mask]
# sal_SSconv = sal_conv[mask]
# depth_SSconv = P[mask]; lat_SSconv = lat[mask]; lon_SSconv = lon[mask]
# mon_SSconv = mon[mask]; day_SSconv = day[mask]; i_SSconv = obs_i[mask].astype(int);
# j_SSconv = (obs_j[mask].astype(int))
# moddepth_SSconv = moddepth[mask]; depthdist_modobs_SSconv = depthdist_modobs[mask]
